In [ ]:
# Устанавливаем дополнительные библиотеки, которых нет в стандартной среде Kaggle
!pip install clearml
!pip install pymorphy2
!pip install nltk

In [ ]:
# Настраиваем окружение ClearML
%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=''
%env CLEARML_API_SECRET_KEY=''

In [ ]:
# Импортируем необходимые библиотеки
import pandas as pd
import csv
from copy import deepcopy
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import re

from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
from sklearn.metrics import f1_score
import numpy as np
from clearml import Task, Logger, Dataset  # Интеграция с ClearML для отслеживания экспериментов и управления датасетами


In [ ]:
#Находим датасет в clear ml и скачиваем его в кэш 
dataset = Dataset.get('2a542c96a0de46b0bc8b4977bf536180')
print(dataset.list_files())

local_path = dataset.get_local_copy() 

In [ ]:
# Читаем обучающую выборку
orig_df = pd.read_csv(f'{local_path}/train.csv',sep=";",quoting=csv.QUOTE_NONE, na_values=['NULL'])

# Пропуски в тексте заменяем пустой строкой, чтобы облегчить дальнейшую обработку
orig_df['text'] = orig_df['text'].fillna('')
orig_df

In [ ]:
# Разбиваем на train/test
train_df, test_df = train_test_split(
    orig_df,
    test_size=0.2,
    random_state=42
)

In [ ]:
# Инициализируем удаленную таску
task = Task.init(
        project_name="sentiment_analysis_kaggle_mllabs",
        task_name="baseline_tfidf_lr",
        task_type="training",
)
task:Task

# Получаем объект логгера, чтобы писать ключевые метрики в ClearML
log = Logger.current_logger()


In [ ]:
# Работаем с копией, чтобы при необходимости вернуться к оригинальному train_df без перезапуска ноутбука
main_train_df = deepcopy(train_df)

In [ ]:
# Выполняем лемматизацию и препроцессинг
patterns = re.compile("[0-9!#$%&'()*+,./:;<=>?[\\]^_`{|}~—\\\"\\-]+")  # Фильтруем цифры и знаки пунктуации, оставляя только полезные токены
stopwords_ru = set(stopwords.words("russian"))  # Ограничиваемся русским списком стоп-слов, чтобы не терять семантику
morph = MorphAnalyzer()  # Морфологический анализатор для нормализации словоформ

_cache = {}

def lemmatize(doc: str) -> str:
    if not isinstance(doc, str):
        return ""
    if doc in _cache:
        return _cache[doc]
    clean = patterns.sub(" ", doc.lower())  # Приводим к нижнему регистру, чтобы TF-IDF не раздувал словарь
    tokens = []
    for token in clean.split():
        if token and token not in stopwords_ru:
            lemma = morph.normal_forms(token)[0]
            tokens.append(lemma)
    res = " ".join(tokens)
    _cache[doc] = res
    return res

main_train_df = deepcopy(train_df)
main_train_df['text'] = main_train_df['text'].apply(lemmatize)
main_train_df.head()


In [ ]:
targets = ['sber', 'vtb', 'gazprom', 'alfabank', 'raiffeisen', 'rshb', 'company']
print(targets)

In [ ]:

first_df = deepcopy(main_train_df)
for target in targets:
    first_df[target] = first_df[target].replace(0,1).replace(-1,1).fillna(0).astype(int)
first_df

In [ ]:
# Обучение детекторов компаний: TF-IDF + LogisticRegression

first_models_by_bank = {}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for target in targets:
    y = first_df[target].astype(int)
    pipe = Pipeline([
        ("tfidf", TfidfVectorizer(ngram_range=(1, 2), max_df=0.9, min_df=2, sublinear_tf=True)),  # Генерируем уни- и биграммы, ограничивая редкие/частые нграммы
        ("clf", LogisticRegression(class_weight="balanced", max_iter=1000, solver="liblinear"))  # Балансируем классы, т.к. негативных примеров значительно меньше
    ])
    param_grid = {"clf__C": [0.5, 1.0, 2.0]}
    grid = GridSearchCV(pipe, param_grid, cv=cv, n_jobs=-1, scoring="f1")
    grid.fit(first_df["text"], y)
    print(f"{target}: best CV f1={grid.best_score_:.3f}, params={grid.best_params_}")
    
    # ClearML logging
    log.report_scalar(title=f"detector_{target}", series="best_f1", value=grid.best_score_, iteration=0)
    log.report_single_value(f"{target} best params: {grid.best_params_}")

    first_models_by_bank[target] = grid.best_estimator_


In [ ]:
# Обучение классификаторов сентимента: TF-IDF + LogisticRegression
second_models_by_bank = {}

for target in targets:
    second_df = main_train_df.loc[main_train_df[target].notnull()][['id','text', target]]
    y = second_df[target].astype(int)
    pipe = Pipeline([
        ("tfidf", TfidfVectorizer(ngram_range=(1, 2), max_df=0.9, min_df=2, sublinear_tf=True)),  # Используем ту же текстовую схему, чтобы сравнение метрик было честным
        ("clf", LogisticRegression(solver='lbfgs', multi_class='multinomial', class_weight='balanced', max_iter=1000))  # Мультином. логистическая регрессия позволяет одновременно предсказывать три класса {-1,0,1}
    ])
    param_grid = {"clf__C": [0.5, 1.0, 2.0]}
    grid = GridSearchCV(pipe, param_grid, cv=cv, n_jobs=-1, scoring="f1_macro") 
    grid.fit(second_df['text'], y)
    print(f"{target}: best CV f1_macro={grid.best_score_:.3f}, params={grid.best_params_}")
    
    # ClearML logging
    log.report_scalar(title=f"sentiment_{target}", series="best_f1_macro", value=grid.best_score_, iteration=0)
    log.report_single_value(f"{target} sentiment best params: {grid.best_params_}")

    second_models_by_bank[target] = grid.best_estimator_


In [ ]:
# Пример для проверки определения банков
text_sample = [lemmatize('RT @alfa_bank: @grey_winged Банк гарантирует отправку смс, опос отвечает за доставку. К сожалению, мы не можем повлиять на сроки решения пр')]
first_models_by_bank["alfabank"].predict(text_sample)


In [ ]:
# Пример для проверки эмоциональной окраски
text_sample = [lemmatize('Опять «порадовал» @sberbank Платёжка получена банком в 15:28 и так и не проведена. Ну никак не хотят работать!')]
second_models_by_bank["sber"].predict(text_sample)

In [ ]:
# Делаем копию тестовой части и прогоняем через тот же препроцессинг, что и train
main_test_df = deepcopy(test_df)
main_test_df['text'] = main_test_df['text'].apply(lemmatize)

for target in targets: 
    # Здесь повторяем бинаризацию меток
    main_test_df[target] = main_test_df[target].replace(0,1).replace(-1,1).fillna(0).astype(int)

main_test_df

In [ ]:
# Финальный инференс: сначала определяем банки, затем определяем тональность
res = []

for _, test_row in main_test_df.iterrows():
    text_sample = [test_row["text"]]
    res_row = {"id": test_row["id"]}

    for target in targets:
        for postfix in ['n','0','p']:
            res_row[f"{target}_{postfix}"] = 0

        det = first_models_by_bank[target].predict(text_sample)[0]
        if det == 1:
            prediction = second_models_by_bank[target].predict(text_sample)[0]
            if prediction == -1:
                res_row[f"{target}_n"] = 1
            elif prediction == 0:
                res_row[f"{target}_0"] = 1
            elif prediction == 1:
                res_row[f"{target}_p"] = 1

    res.append(res_row)

all_cols = [f"{t}_{p}" for t in targets for p in ['n','0','p']]
pred_test_df = pd.DataFrame(res)
pred_test_df = pred_test_df.merge(main_test_df[['id']], on='id', how='right')  # merge гарантирует сохранение порядка id
pred_test_df = pred_test_df[['id'] + all_cols]
Y_pred = pred_test_df[all_cols].fillna(0).astype(int).values

# Формируем Y_true из оригинальных меток в test_df
y_true_rows = []
for _, r in test_df.iterrows():
    row = {c: 0 for c in all_cols}
    for target in targets:
        val = r[target]
        if pd.isna(val):
            continue
        if val == -1:
            row[f"{target}_n"] = 1
        elif val == 0:
            row[f"{target}_0"] = 1
        elif val == 1:
            row[f"{target}_p"] = 1
    y_true_rows.append(row)

y_true_df = pd.DataFrame(y_true_rows)
y_true_df = pd.concat([test_df[['id']].reset_index(drop=True), y_true_df], axis=1)
y_true_df = y_true_df.merge(pred_test_df[['id']], on='id', how='right')
Y_true = y_true_df[all_cols].fillna(0).astype(int).values

col_f1 = [f1_score(Y_true[:, i], Y_pred[:, i], zero_division=0) for i in range(len(all_cols))]  # Считаем метрику по каждому «банк+тональность»
test_macro_f1_21 = float(np.mean(col_f1))  # Сводим в усреднённую метрику

log.report_scalar(title="test_macro_f1_21cols", series="macro_f1", value=test_macro_f1_21, iteration=0)
log.report_text(f"Per-column F1 (test): {dict(zip(all_cols, [float(x) for x in col_f1]))}")


In [ ]:
# Строим столбчатую диаграмму F1 по каждому признаку и логгируем в ClearML
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.bar(all_cols, col_f1)
plt.xticks(rotation=45, ha='right')
plt.ylim(0, 1)
plt.title('F1 per column (test)')
plt.tight_layout()
log.report_matplotlib_figure(title='test_f1', series='f1_bar', iteration=0, figure=plt.gcf())  # Логируем график в ClearML
plt.close()

In [ ]:
# Закрываем задачу, чтобы ClearML корректно сохранил артефакты
task.close()